In [9]:
# Import Libraries

import os

# Analysis

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio

# Visualizations

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import contextily

import warnings
warnings.filterwarnings('ignore') # Ignores some warnings

In [2]:
# Load raster template

raster_path = os.path.join(os.getcwd(), '..', '2_Model_Pollutant_Exposure', 'template.npy')

raster = np.load(raster_path)

# Boundary

mpls_path = os.path.join(os.getcwd(), '..', '1_Data_IO', 'Data', 'mpls_boundary.geojson')
mpls = gpd.read_file(mpls_path)

In [3]:
# Load Interpolated PurpleAir

purple_air_path = os.path.join(os.getcwd(), '..', '4_Validate_Index_Surface',
                               'PurpleAir_Interpolation_Normalized.tif')

rasterio_rast = rasterio.open(purple_air_path)

PurpleAir = rasterio_rast.read(1)

rasterio_rast.close()

In [4]:
# Load calculated stats

stats_df = pd.read_csv('Sensitivity_Stats.csv')

In [5]:
# Let's check the best models?

bestrmse_filename = stats_df.iloc[stats_df.PurpleAir_RMSE_mpls.argmin()].Hazard_Index_Filename
other_good_ones = ['15.0sig_45I-55T_HazardIndex.tif',
                   '14.5sig_40I-60T_HazardIndex.tif',
                   '14.5sig_45I-55T_HazardIndex.tif',
                   '14.0sig_40I-60T_HazardIndex.tif',
                   '14.0sig_45I-55T_HazardIndex.tif']
curiousities = ['15.0sig_60I-40T_HazardIndex.tif',
               '14.5sig_60I-40T_HazardIndex.tif',
               '14.0sig_60I-40T_HazardIndex.tif']

filenames = [bestrmse_filename] + other_good_ones + curiousities

In [6]:
# Load rasterized indices

# Filenames

path_to_tests = os.path.join(os.getcwd(), 'Hazard_Indices')

layernames = []

tests = {}

for filename in filenames:

    path = os.path.join(path_to_tests, filename)
    
    layername = filename[:-4]
    
    rasterio_rast = rasterio.open(path)
    
    tests[layername] = rasterio_rast.read(1)
    
    rasterio_rast.close()

In [7]:
# Check out averages in the various classes

interesting_columns = ['Hazardous_IndexMean',
                       'Definitely Declining_IndexMean',
                       'Still Desirable_IndexMean',
                       'Best_IndexMean',
                       'Open Water_IndexMean',
                       'Business and Industrial_IndexMean',
                       'Uncertain_IndexMean',
                       'Undeveloped_IndexMean',
                       'Park / Open Space_IndexMean'
                      ]

HSG_codes = ['Hazardous', 'Definitely Declining', 'Still Desirable',
             'Best', 'Open Water', 'Business and Industrial', 'Uncertain', 
             'Undeveloped', 'Park / Open Space']

    

In [20]:
# PLOT!!!!!!!

# This custom formatter for contour lines removes trailing zeros, e.g. "1.0" becomes "1".

def fmt(x):
    s = f"{x:.1f}"
    if s.endswith("0"):
        s = f"{x:.0f}"
    return rf"{s}" if plt.rcParams["text.usetex"] else f"{s}"

for filename in filenames:
    
    testname = filename[:-4]
    
    # Plot the zonal stats
    zonalstats_filename = testname + '_Zonal_Stats.geojson'
    zonalstats_path = os.path.join(os.getcwd(), 'Zonal_Stats', zonalstats_filename)
    zonal_stats = gpd.read_file(zonalstats_path)
    zonal_stats_clipped = gpd.clip(zonal_stats, mpls)
    # Plot the zonal mean
    f, ax = plt.subplots(1,1,figsize=(16,16))
    zonal_stats_clipped.plot('mean', legend = True, cmap = 'afmhot_r', ax = ax)
    ax.set_axis_off()
    plt.savefig('BestModelVis/' + testname + ' Zonal Statistics.png', dpi = 300)
    plt.close()
    
    # Plot the cost index
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    Z = tests[testname]
    
    f, axs = plt.subplots(2, 1, figsize = (16, 16), height_ratios=[1, 6], width_ratios = [1])
    
    f.suptitle(testname, fontsize = 24)
    
    sns.histplot(Z.flatten(), ax = axs[0])

    art = axs[1].pcolormesh(raster[0], raster[1], Z.T,
                            shading='auto', cmap = 'afmhot_r',
                            norm = norm,
                            alpha = 0.4) # Plot residual
    
    CS = axs[1].contour(raster[0], raster[1], Z.T, levels = 5, norm = norm, cmap='afmhot_r', alpha = 0.5) # Contours

    # Label contours

    axs[1].clabel(CS, CS.levels, inline=True, fmt=fmt, fontsize=10, colors = 'black')
    mpls.plot(ax = axs[1],
             edgecolor='black', facecolor='none',
             linewidth=3,
             zorder = 2,
             legend = True)
    
    axs[1].margins(x=-0.2, y=-0.2)
    f.colorbar(art, ax = axs[1]) # Add colorbar
    axs[1].set_axis_off()

    # Add basemap

    contextily.add_basemap(ax = axs[1], 
                           crs = 'EPSG:26915', 
                           source=contextily.providers.CartoDB.Voyager
                          ) 
    plt.savefig('BestModelVis/' + testname + '.png', dpi = 300)
    plt.close()
    
    # Plot the Residual
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)
    dif = PurpleAir - tests[testname]
    rmse = stats_df[stats_df.Hazard_Index_Filename == filename].PurpleAir_RMSE_mpls.values[0]
    
    f, axs = plt.subplots(2, 1, figsize = (16, 16), height_ratios=[1, 6], width_ratios = [1])
    
    f.suptitle(testname + ' Residuals', fontsize = 24)
    
    sns.histplot(dif.flatten(), ax = axs[0])

    art = axs[1].pcolormesh(raster[0], raster[1], dif.T,
                            shading='auto', cmap = 'RdBu',
                            norm = norm,
                            alpha = 0.4) # Plot residual
    CS = axs[1].contour(raster[0], raster[1], dif.T, levels = 5, norm = norm, cmap='RdBu', alpha = 0.5) # Contours

    # Label contours

    axs[1].clabel(CS, CS.levels, inline=True, fmt=fmt, fontsize=10, colors = 'black')
    mpls.plot(ax = axs[1],
             edgecolor='black', facecolor='none',
             linewidth=3,
             zorder = 2,
             legend = True)
    
    axs[1].margins(x=-0.2, y=-0.2)
    f.colorbar(art, ax = axs[1]) # Add colorbar
    title = 'Purple Air Interpolation \n RMSE for Minneapolis = ' + str(rmse)
    axs[1].set_title(title) # Add title
    axs[1].set_axis_off()

    # Add basemap

    contextily.add_basemap(ax = axs[1], 
                           crs = 'EPSG:26915', 
                           source=contextily.providers.CartoDB.Voyager
                          ) 
    plt.savefig('BestModelVis/' + testname + ' Residuals.png', dpi = 300)
    plt.close()
    
    # Shifted Residual
    
    f, axs = plt.subplots(2, 1, figsize = (16, 16), height_ratios=[1, 6], width_ratios = [1])
    
    f.suptitle(testname + ' Residuals', fontsize = 24)
    
    sns.histplot(dif.flatten(), ax = axs[0])

    art = axs[1].pcolormesh(raster[0], raster[1], dif.T,
                            shading='auto', cmap = 'RdBu',
                            #vmin=-1, vmax=1,
                            alpha = 0.4) # Plot residual
    mpls.plot(ax = axs[1],
         edgecolor='black', facecolor='none',
         linewidth=3,
         zorder = 2,
         legend = True)
    
    axs[1].margins(x=-0.2, y=-0.2)
    f.colorbar(art, ax = axs[1]) # Add colorbar
    title = 'Purple Air Interpolation \n RMSE for Minneapolis = ' + str(rmse)
    axs[1].set_title(title) # Add title
    axs[1].set_axis_off()

    # Add basemap

    contextily.add_basemap(ax = axs[1], 
                           crs = 'EPSG:26915', 
                           source=contextily.providers.CartoDB.Voyager
                          ) 
    plt.savefig('BestModelVis/' + testname + ' Residuals_shifted.png', dpi = 300)
    plt.close()
    
    # Plot HSG_SCALE and Index Mean by SCALE
    
    stats = stats_df[stats_df.Hazard_Index_Filename == filename]
    index_means = stats[interesting_columns].values[0]
    
    column_to_stat_map = dict(zip(HSG_codes, index_means))
    
    f, ax = plt.subplots(1,1, figsize = (16,16))
    zonal_stats = gpd.read_file(zonalstats_path)
    zonal_stats['IndexMean'] = zonal_stats.HSG_SCALE.apply(lambda x: column_to_stat_map[x])    
    # zonal_stats['label'] = zonal_stats.HSG_SCALE.apply(lambda x: x + 'Index Mean = '
    #                                                   ).str.cat(zonal_stats.IndexMean.to_string())
    # zonal_stats.HSG_SCALE.str.cat(zonal_stats.IndexMean.to_string(), sep = 'Index Mean = ')
    zonal_stats_clipped = gpd.clip(zonal_stats, mpls)
    zonal_stats_clipped.plot('IndexMean', cmap = 'afmhot_r', categorical = True, 
                     legend = True, ax = ax)
    
    # Change Legend
    leg = ax.get_legend()
    inv_column_to_stat_map = {v: k for k, v in column_to_stat_map.items()}
    
    for text in leg.get_texts():
        old_text = text.get_text()[:5] # 5 sig figs
        new_text = inv_column_to_stat_map[float(text.get_text())] + ' Mean Index = ' + old_text
        text.set_text(new_text)
    
    # Relocate legend
    leg.set_bbox_to_anchor((1.25,1))
    ax.set_title('IndexMean by Category')
    ax.set_axis_off()
    plt.savefig('BestModelVis/' + testname + ' Index Mean.png', dpi = 300)
    plt.close()

In [ ]:
best_stats = stats_df[stats_df.Hazard_Index_Filename == filename]